<a href="https://colab.research.google.com/github/thdgmltjd123/Assignment01/blob/master/Assignment09_20150711.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#구글 드라이브와 코랩 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import transforms, datasets

In [25]:
transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.1307,),(0.3081,)),
])

In [26]:
#부모클래스 nn.Module로부터 상속받는 자식클래스 classification
class classification(nn.Module):
  def __init__(self):
    #부모클래스의 생성자를 이용하겠다.
    super(classification, self).__init__()

    #layer 1층 : 28*28 데이터가 20*20으로 나옴
    #activation_function = Sigmoid
    self.classifier1 = nn.Sequential(
        nn.Linear(in_features = 28*28, out_features=20*20),
        nn.Sigmoid(),
    )
    #layer 2층 : 20*20 데이터가 10*10으로 나옴
    #activation_function = Sigmoid
    self.classifier2 = nn.Sequential(
        nn.Linear(in_features = 20*20, out_features = 10*10),
        nn.Sigmoid(),
    )
    #output layer 3층 : 10*10 데이터가 10으로 나옴
    #activation_function = LogSoftmax
    self.classifier3 = nn.Sequential(
        nn.Linear(in_features=10*10, out_features = 10),
        nn.LogSoftmax(dim = 1),
    )
  #순전파 알고리즘
  def forward(self, inputs):
    x = inputs.view(inputs.size(0),-1)
    x = self.classifier1(x)
    x = self.classifier2(x)
    out = self.classifier3(x)

    return out

In [49]:
#fully-connected Neural Network using Pytorch
def fc_NN(train_loader, test_loader, epoch, learning_rate, no_cuda = False):
  use_cuda = not no_cuda and torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  creterion = nn.NLLLoss()
  Model = classification().to(device)
  optimizer = torch.optim.SGD(Model.parameters(), lr = learning_rate)
  train_L_iters = []
  train_acc_iters = []
  test_L_iters = []
  test_acc_iters = []
  for i in range(1,epoch+1):
    # Train mode
    train_loss = 0
    train_correct = 0
    Model.train()
    for batch_idx,(data, target) in enumerate(train_loader):
      #data와 target을 gpu를 사용할 수 있게 매핑
      data, target = data.to(device), target.to(device)
      # backward하는 과정에서 기존의 gradient값에 새로운 gradient 값을
      # 더해주기때문에 grad값을 매 backward마다 0으로 초기화해줘야 올바른 학습이 가능
      optimizer.zero_grad()
      # 순전파
      output = Model(data)
      # reduction = (default) "mean"
      loss = F.nll_loss(output,target)
      train_loss += F.nll_loss(output,target,reduction = 'sum').item()
      pred = output.argmax(dim = 1, keepdim = True)
      train_correct += pred.eq(target.view_as(pred)).sum().item()
      # 역전파
      loss.backward()
      optimizer.step()
    # 에포크별 평균 트레인 로스 계산
    train_loss /= len(train_loader.dataset)
    train_L_iters.append(train_loss)
    # 에포크별 평균 정확도 계산
    accuracy =  100 * train_correct / len(train_loader.dataset)
    train_acc_iters.append(accuracy)
    if i % 10 == 0:
      print("progress = {}, current loss = {}, current device = {}".format(100 * i / epoch, train_loss, torch.cuda.get_device_name()))    
    # Test Mode
    Model.eval()
    test_loss = 0
    test_correct = 0
    #torch.no_grad() 테스트 할 때, 위에서 계산한 grad를 더이상 업데이트 하지 않음.
    with torch.no_grad():
      for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = Model(data)
        test_loss += F.nll_loss(output, target, reduction = 'sum').item()
        pred = output.argmax(dim = 1, keepdim = True)
        test_correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_L_iters.append(test_loss)
    accuracy = 100 * test_correct / len(test_loader.dataset)
    test_acc_iters.append(accuracy)

  return train_L_iters, train_acc_iters, test_L_iters, test_acc_iters
    

In [ ]:
batch_size = [32,64,128]
epoch = 100
learning_rate_value = 0.01
no_cuda = False
batch_train_loss = []
batch_test_loss = []
batch_train_accuracy = []
batch_test_accuracy = []
data_path = '/content/drive/My Drive/Colab Notebooks'
for b in batch_size:
  print("batch_size  = {}".format(b))
  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root = data_path, train = True, 
                    download=True, transform = transform),
                    batch_size = b,
                    shuffle = False)
  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root = data_path, train = False,
                    download=True, transform = transform),
                    batch_size = b,
                    shuffle = False)
  train_L_iters, train_acc_iters,test_L_iters, test_acc_iters = fc_NN(train_loader, test_loader,epoch, learning_rate_value)
  batch_train_loss.append(train_L_iters)
  batch_train_accuracy.append(train_acc_iters)
  batch_test_loss.append(test_L_iters)
  batch_test_accuracy.append(test_acc_iters)

batch_size  = 32


# **[Output] Plot the Training and testing losses with a batch size of 32**

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(batch_train_loss[0], c = "red", label = "train loss")
plt.plot(batch_test_loss[0], c = "blue", label = "test loss")
plt.title("loss at batch_size = 32")
plt.legend(loc = "best")
plt.show()

# **[Output] Plot the training and testing accuracies with a batch size of 32**

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(batch_train_accuracy[0], c = "red", label = "train accuracy")
plt.plot(batch_test_accuracy[0], c = "blue", label = "test accuracy")
plt.title("accuracy at batch_size = 32")
plt.legend(loc = "best")
plt.show()

# **[Output] Plot the training and testing losses with a batch size of 64**

In [31]:
torch.cuda.get_device_name()

'Tesla T4'

# **[Output] Plot the training and testing accuracies with a batch size of 64**

# **[Output] Plot the training and testing losses with a batch size of 128**

tensor(2.3649, grad_fn=<NllLossBackward>)

# **[Output] Plot the training and testing accuracies with a batch size of 128**

# **[Output] Print the loss at convergence with different mini-batch sizes**

# **[Output] Print the accuracy at convergence with different mini-batch sizes**